<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Aug 24th, 2021</h5>

### Step 0: Import Python packages and initiate koster database

In [1]:
# Save your user name, password and Zooniverse project number.
import getpass
zoo_user = getpass.getpass('Enter your Zooniverse user')
zoo_pass = getpass.getpass('Enter your Zooniverse password')
project_n = getpass.getpass('Enter the number of the Zooniverse project')

Enter your Zooniverse user········
Enter your Zooniverse password········
Enter the number of the Zooniverse project········


In [2]:
# Import required modules
import sys
sys.path.append('..')
import utils.t12_utils as t12

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --user $zoo_user --passw $zoo_pass --project_n $project_n

Broken libmagic installation detected. The python-magic module is installed but can't be imported. Please check that both python-magic and the libmagic shared library are installed correctly. Uploading media other than images may not work.


There is no initial folder with initial information about the sites, movies and species.
 Please enter the ID of a Google Drive zipped folder with the inital database information. 
 For example, the ID of the template information is: 1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
ID of Google Drive zipped folder········
Retrieving the file from  https://drive.google.com/uc?id=11XQIKdwX9wHrzOcrDlRrs9Oq5uA2o95k


Downloading...
From: https://drive.google.com/uc?id=11XQIKdwX9wHrzOcrDlRrs9Oq5uA2o95k
To: C:\Users\Victor\koster_data_management\tutorials\db_csv_info.zip
100%|██████████| 15.3k/15.3k [00:00<?, ?B/s]


Updated sites
Updated movies
Updated species
Retrieving information from subjects uploaded to Zooniverse
Updated subjects


### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

In [3]:
# TODO Display a selectable list of workflow ids with their names and a list of versions of the workflow of interest
#workflow_id, workflow_version, subj_type = t13.get_workflows(zoo_user, zoo_pass)
    
# Temporary example objects
workflow_id = 17719
workflow_version = 0.0
subj_type = "clip"

# Retrieve classifications from the workflow of interest
total_df, class_df = t12.get_classifications(workflow_id, workflow_version, subj_type, zoo_user, zoo_pass)

TypeError: auth_session() missing 1 required positional argument: 'project_n'

### Step 2: Aggregate classifications received on the workflow of interest

In [ ]:
# TODO Display a selectable list of min number of users, agreement, 
#agg_user, min_users = t13.set_aggregation_parameters()

# Specify the agreement threshold required among cit scientists
agg_users = 0.8
min_users = 3

class_df1 = t12.aggregrate_classifications(class_df, subj_type, agg_users, min_users)

### Step 3: Summarise the number of classifications based on the agreement specified

### Step 4: Use the subject explorer widget to get more information about specific subjects and their aggregated classifications

In [ ]:
# Get all classified subjects from specified workflows
total_df, class_df = t13.get_exports(user, password)

clips_df = t13.process_clips(total_df, class_df, clips_workflow_id, clips_workflow_version)
frames_df = t13.process_frames(db_path, total_df, frames_workflow_id,
                               frames_workflow_version, 
                               "https://drive.google.com/file/d/1z72CqTtEBtqk6936H1YNrCjc5NRopF0g/view?usp=sharing")

t13.launch_viewer(total_df, clips_df, frames_df)

In [ ]:
# END